
### 任务与执行策略之间的隐性耦合
  Executor框架可以将提交的任务和任务的执行策略解耦合，但是并非所有的任务使用所有的执行策略。<br/>
* 依赖性任务。提交的任务不是相互独立的，而是需要依赖其他的任务，则应该小心选择执行策略，避免产生活跃性问题。
* 使用线程封闭机制的任务。依赖在线程封闭的线程池（如单线程）的线程池中执行的任务，一旦将环境改为多线程，则会失去线程安全性。
* 对响应时间敏感的线程。
* 使用TreadLocal的任务。TreadLocal变量使每个线程拥有一个变量的私有版本，只要条件允许，Executor可以自由的重用这些线程。但是线程会在适当时候被回收，或者线程抛出运行时异常，会被一个新的工作线程取代。所以只有当TreadLocal保存的对象是受限于线程的生命周期时，才应该被使用。并且线程池的线程中不该使用TreadLocal在任务之间传递值（如果是可复用线程，则应该在每次把线程返回给线程池时，清除掉该ThreadLocal对象）



#### 线程饥饿死锁
* 依赖性任务提交到同一个线程池，通常会引发死锁。如，线程A提交了线程B在同一个线程池，并等待他的执行结果。
* 更大的线程池中，如果正在执行任务的线程都由于等待其他仍然处于工作队列中的线程所阻塞，这种现象称为**饥饿死锁**。

####  运行时间较长的任务
 *  如果任务阻塞时间较长，那么不出现死锁，线程响应性也会很糟糕，会造成线程池阻塞，甚至增加执行时间较短的任务的时间。
 *  如果线程池中线程数量远小于长时间执行任务的数量，到最后可能造成所有线程都在执行这些线程，从而影响整体的响应性。
 以下技术可以缓解
 *  限定任务等待资源的时间，不要无限制等待。在阻塞方法上设置等待时间，超时后标志失败，终止任务或将任务重新放回队列随后执行。
 *  线程池中总是充满阻塞的任务，也说明线程池过小。

### 设置线程池的大小
 线程池理想大小取决于被提交任务的类型，及所部署的特性。通常不会固定配置。通过配置机制，或根据RunTime。availableProcessor来动态计算。<br/>
 避免线程池过大或过小。过小会造成并发性不足，降低任务执行效率吧，过大会造成过多线程在CPU上竞争，降低吞吐率。<br/>
 线程池配置规则如下:
 * 计算密集型任务：线程池大小为N<sub>cpu</sub>+1
 * 包含I/O以及其他阻塞操作的任务：设置的更大，因为线程不会一直执行。估算出等待时间和执行时间的壁纸。

###  配置TreadPoolExecutor
ThreadPoolExecutor提供了一些基本的线程池的实现，若不满足要求，可以调用TreadPoolExecutor的构造函数，定制自己配置参数的的线程池。
#### 线程的创建与销毁
线程池的基本大小，最大大小，以及存活时间等因素共同负责线程的创建和销毁。
* 线程池的基本大小：线程池目标大小，没有任务执行使，线程池内线程数量。只有在工作队列满了的时候，才会创建超出这个数量的线程。
* 线程池的最大大小：可同时活动的线程的最大数量的上限。
* 线程回收：线程空闲时间超过了存活时间时，会被标记为可回收的，且当线程池当前大小超过线程池最大大小时，该线程会被终止。
* newFixedThreadPool 将线程池的基本大小和最大大小设置为参数中指定的值，不设置超时时间
* newCachedThreadPool 将线程池的最大大小设为Integer.MAX_VALUE。基本大小为0，超时时间为1分钟——可无限扩展，且需求降低时会自动收缩。

#### 管理任务队列
在有限的线程池中会限制可并发执行的数量（如单线程线程池，不存在任务并发）
* 无限创建线程会导致不稳定性。采用固定大小的线程池可以避免。但是高负载情况下仍有可能耗尽资源，若心情求速度超过了线程池处理速度，新来的请求将会被存储在一个由线程池维护的Runnable队列中，而不会去竞争CPU资源。但是客户提交请求速率远高于线程处理速率，依然会耗尽资源。
* 即使请求到达的平均速率稳定，仍有可能突然请求突增，且在耗尽内存之前，响应性能会随着队列长度增加而越来越糟。
* TreadPoolExecutor允许提交一个BlockingQueue来保存等待执行的任务。基本任务排队方法有：无界队列，有界队列，同步移交。
* newFixedThreadPool与newSingleThreadExecutor默认使用无界的LinkedBlockingQueue——如果线程池中所有线程都处于忙碌状态，新到达的线程会被加入队列，并且当任务持续到达，队列会持续无限增大。
* 有界队列可以防止资源耗尽的情况，如ArrayBlockQueue，有界的LinkedBlockingQueue，有界的PriorityBlockingQueue。但是要求队列大小与线程池大小同步调节，比如线程池小而队列大，则会降低吞吐量，却有助于减少内存的使用量以及上下文的切换。
* 对于无限大（非常大）的线程池，适用于SynchronousQueue避免任务排队。SynchronousQueue不是任务队列，而是一种在线程间移交的方式。一个元素放入一个元素，必须有另一个线程等待接收。若线程池大小小于最大值且没有线程在等待，则该线程被提交，否则根据饱和策略被拒绝。只有线程池无限大，或者可拒绝任务时，SynchronousQueue才有意义。否则应该把线程直接提交给线程池。
* newCachedThreadPool使用适用于SynchronousQueue避免任务排队
* 先进先出队列，优先队列
* 默认线程池：newCacheThreadPool：提供了更好的排队性能。需要固定大小的线程池不使用，容易发生过载问题。
#### 饱和策略

队列填满之后，饱和策略开始发挥作用。ThreadPoolExecutor可以采用setRejectedExecutionHandler来设置，JDK提供了以下几种饱和策略：
* AbortPolicy——默认策略，抛出未检查的RejectedExecutionException
* CallerRunsPolicy——将某些任务退回给调用者，从而降低新任务的流量。
* DiscardOldestPolicy——抛弃最老的

#### 线程工厂
* 通过扩展ThreadFactory 自定义线程工厂。
* 需要安全策略进行对某些特殊代码库的访问，可以扩展Executors.PrivilegedThreadFactory
#### 调用构造函数后定制ThreadPoolExecutor
* 通过seter函数来定制
* lExecutors中工厂方法创建的除外，这种需要转型为ThreadPoolExecutor来设置
* 不希望线程池被定制，则使用Executors.unconfigurableExecutorService包装器来包装线程池

### 扩展ThreadPoolExecutor
ThreadPoolExecutor提供了可以在子类中改写的方法：
* beforeExecute——可以添加日志，计时，监视，统计信息收集的功能。
* afterExecute——可以添加日志，计时，监视，统计信息收集的功能。
* 任务执行中抛出异常 会执行afterExecute，beforeExecute抛出异常，直接终止，不执行afterExecute
* terminated——释放线程池使用的资源。


In [ ]:
### 递归算法的并行化

